# This notebook demonstrates accessing and visualizing scan and segmentations from planC

In [1]:
import oct2py

In [2]:
# Without using Octave magic
#oc = oct2py.Oct2Py(temp_dir=oc_temp_dir)
#oc.version()

## Define directory to store temporary variables for I/O between Octave and Python
### Select a location that has fast disk read/write access

In [3]:
oc_temp_dir="C:/software/octave_temp_dir"

In [4]:
import os
if not os.path.isdir(oc_temp_dir):
    os.mkdir(oc_temp_dir)

## Load Octave cell magic

In [5]:
%load_ext oct2py.ipython

## Add CERR to Octave path

In [6]:
%%octave -t {oc_temp_dir}
cerrPath = 'C:/software/CERR_octave_dev/CERR'
addpath(genpath(cerrPath));
warning('off')
eval('pkg load io');
eval('pkg load image');
eval('pkg load statistics');

cerrPath = C:/software/CERR_octave_dev/CERR

In [7]:
## Extract scan and segmentation masks, make them available to Python

Loacation of sample dataset

In [8]:
%%octave -t {oc_temp_dir}
sampleData = fullfile(getCERRPath(),'..','Unit_Testing','data_for_cerr_tests', ...
              'CERR_plans','head_neck_ex1_20may03.mat.bz2');
planC = loadPlanC(sampleData, tempdir());
planC = updatePlanFields(planC);
planC = quality_assure_planC(sampleData, planC);
indexS = planC{end};

% Structure names
planC{indexS.structures}.structureName

%Get scan array
indexS = planC{end};
scanNum = 1;
ctOffset = planC{indexS.scan}(scanNum).scanInfo(1).CTOffset;
scanArray = single(getScanArray(scanNum,planC)) - ctOffset;

%Get structure labels & masks
numStructs = length(planC{indexS.structures});
structNameC = {planC{indexS.structures}.structureName};

strNameC = {'Brain', 'Brain Stem', 'Parotid Gland',...
              'Spinal Cord', 'Target 1', 'Target 2'};
for strNum = 1:length(strNameC)
    strName = strNameC{strNum};
    idx = getMatchingIndex(strName,structNameC,'EXACT');
    mask3M = getStrMask(idx, planC);
    maskC{strNum} = mask3M;
end

% Get scan grid
[xScanValsV,yScanValsV,zScanValsV] = getScanXYZVals(planC{indexS.scan}(scanNum));

CERR>>  Decompressing head_neck_ex1_20may03.mat.bz2...

7-Zip 4.65  Copyright (c) 1999-2009 Igor Pavlov  2009-02-03

Processing archive: C:\software\CERR_octave_dev\CERR\CERR_core\..\Unit_Testing\data_for_cerr_tests\CERR_plans\head_neck_ex1_20may03.mat.bz2

Extracting  head_neck_ex1_20may03.mat

Everything is Ok

...

ans = SKIN

ans = Spinal Cord

ans = Target 1

ans = Target 2

ans = Target 3

Push planC from Python to Octave

Pull binary masks from Octave to Python

In [9]:
%octave_pull maskC strNameC scanArray xScanValsV yScanValsV zScanValsV

## Visualize segmentation on orthogonal views

In [10]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import ipywidgets as widgets
from ipywidgets import Layout

#dx, dy = 1, 1
#x = np.arange(0, 127, dx)
#y = np.arange(0, 100, dy)
#extent = np.min(x), np.max(x), np.min(y), np.max(y)

window_center = 45
window_width = 1250

extent_trans = np.min(xScanValsV), np.max(xScanValsV), np.min(yScanValsV), np.max(yScanValsV)
extent_sag = np.min(yScanValsV), np.max(yScanValsV), np.min(zScanValsV), np.max(zScanValsV)
extent_cor = np.min(xScanValsV), np.max(xScanValsV), np.min(zScanValsV), np.max(zScanValsV)

clear_output(wait=True)

def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    
    return window_image

def rotate_image(img):
    return(list(zip(*img)))

def show_slice(slcNum, view):
    clear_output(wait=True)
    print(view + ' view slice ' + str(slcNum))
    if 'fig' in locals():
        fig.remove()
    fig, (ax,ax_legend) = plt.subplots(1,2)
    ax_legend.set_visible(False)
    #window_center = 45
    #window_width = 1250
    
    if view.lower() == 'axial':
        windowed_img = window_image(scanArray[: ,: ,slcNum - 1], window_center, window_width)
        extent = extent_trans
    elif view.lower() == 'sagittal':
        #windowed_img = rotate_image(window_image(scanArray[40:210, slcNum + 79, :], window_center, window_width))
        windowed_img = rotate_image(window_image(scanArray[:, slcNum - 1, :], window_center, window_width))
        extent = extent_sag
    elif view.lower() == 'coronal':
        windowed_img = rotate_image(window_image(scanArray[slcNum - 1, :, :], window_center, window_width))
        extent = extent_cor
    else:
        raise Exeception('Invalid view type: ' + view)
    
    im1 = ax.imshow(windowed_img, cmap=plt.cm.gray, alpha=1,
                    interpolation='nearest', extent=extent)
    
    cmaps = [plt.cm.get_cmap("Oranges").copy(),plt.cm.get_cmap("Oranges").copy(), \
             plt.cm.get_cmap("Blues").copy(),plt.cm.get_cmap("Blues").copy(), \
             plt.cm.get_cmap("Purples").copy(),plt.cm.get_cmap("Greens").copy()]

    if view.lower() == 'axial':
        for maskNum in range(0,6,1):
            mask_cmap = cmaps[maskNum]
            mask_cmap.set_under('k', alpha=0)
            im2 = ax.imshow(maskC[0,maskNum][:,:,slcNum-1], 
                            cmap=mask_cmap, alpha=.8, extent=extent,
                            interpolation='none', clim=[0.5, 1])      
            
    elif view.lower() == 'sagittal': 
        for maskNum in range(0,6,1):
            mask_cmap = cmaps[maskNum]
            mask_cmap.set_under('k', alpha=0)
            #im2 = ax.imshow(rotate_image(maskC[0,maskNum][50:200, slcNum + 79, :]),
            #                cmap=mask_cmap, alpha=.8, extent=extent,
            #                interpolation='none', clim=[0.5, 1])
            im2 = ax.imshow(rotate_image(maskC[0,maskNum][:, slcNum - 1, :]),
                            cmap=mask_cmap, alpha=.8, extent=extent,
                            interpolation='none', clim=[0.5, 1])
            
    elif view.lower() == 'coronal':
        for maskNum in range(0,6,1):
            mask_cmap = cmaps[maskNum]
            mask_cmap.set_under('k', alpha=0)
            #im2 = ax.imshow(rotate_image(maskC[0,maskNum][slcNum + 60, 50:200, :]),
            #                cmap=mask_cmap, alpha=.8, extent=extent,
            #                interpolation='none', clim=[0.5, 1])
            im2 = ax.imshow(rotate_image(maskC[0,maskNum][slcNum - 1, :, :]),
                            cmap=mask_cmap, alpha=.8, extent=extent,
                            interpolation='none', clim=[0.5, 1])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    plt.rcParams["figure.figsize"] = (5, 3)
    plt.show()
    
slice_slider_axial = widgets.IntSlider(min=1,max=79,step=1)
outputSlc_axial = widgets.Output()
    
#slice_slider_sagittal = widgets.IntSlider(min=1,max=88,step=1)
slice_slider_sagittal = widgets.IntSlider(min=1,max=256,step=1)
outputSlc_sagittal = widgets.Output()

#slice_slider_coronal = widgets.IntSlider(min=1,max=116,step=1)
slice_slider_coronal = widgets.IntSlider(min=1,max=256,step=1)
outputSlc_coronal = widgets.Output()

display(slice_slider_axial, outputSlc_axial, slice_slider_sagittal, outputSlc_sagittal, slice_slider_coronal,outputSlc_coronal)

def update_slice_axial(change):
    with outputSlc_axial:
        show_slice(change['new'], 'axial')

def update_slice_sagittal(change):
    with outputSlc_sagittal:
        show_slice(change['new'], 'sagittal')
        
def update_slice_coronal(change):
    with outputSlc_coronal:
        show_slice(change['new'], 'coronal')
        
slice_slider_axial.observe(update_slice_axial, names='value')
slice_slider_sagittal.observe(update_slice_sagittal, names='value')
slice_slider_coronal.observe(update_slice_coronal, names='value')

# Set values to display figures
slice_slider_axial.value = 40
slice_slider_sagittal.value = 120
slice_slider_coronal.value = 120

IntSlider(value=1, max=79, min=1)

Output()

IntSlider(value=1, max=256, min=1)

Output()

IntSlider(value=1, max=256, min=1)

Output()